# 1. Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import tensorflow as tf
import keras
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from keras.utils import to_categorical
import seaborn as sns
from sklearn.compose import ColumnTransformer
from statistics import stdev
from keras import regularizers
from sklearn.metrics import classification_report
from sklearn.svm import SVC

Using TensorFlow backend.


#  2. Data Preprocessing

In [2]:
df_raw = pd.read_csv("../input/crystal-system-properties-for-liion-batteries/lithium-ion batteries.csv")
data = df_raw.drop(['Materials Id'], axis=1)

numerical_transformer = StandardScaler()
label_transformer = OrdinalEncoder()
n_cols = [c for c in data.columns if data[c].dtype in ['int64', 'float64', 'int32', 'float32']]
obj_cols = [c for c in data.columns if data[c].dtype in ['object', 'bool']]

ct = ColumnTransformer([('num', numerical_transformer, n_cols), ('non_num', label_transformer, obj_cols),])
processed = ct.fit_transform(data)
new_data = pd.DataFrame(columns=data.columns, data=processed)
new_data.head()
target_names_all =[item[0] for item in set([(a,b) for a,b in zip(data['Crystal System'].values, new_data['Crystal System'].values)])]

In [3]:
train_data, test_data = train_test_split(new_data, test_size=0.2, shuffle=True, random_state=8)
X_train = np.array(train_data.drop(['Crystal System'], axis=1))
y_train = np.array(train_data['Crystal System'])
X_test = np.array(test_data.drop(['Crystal System'], axis=1))
y_test = np.array(test_data['Crystal System'])

# 3. DNN

In [4]:
y_encoded = to_categorical(y_train)
model = Sequential()
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='softsign', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_encoded, epochs=100, batch_size=16, validation_split=0.1)

Train on 243 samples, validate on 28 samples
Epoch 1/100
243/243 [==============================] - 1s 5ms/step - loss: 12.0553 - accuracy: 0.4362 - val_loss: 10.9907 - val_accuracy: 0.1429
Epoch 2/100
243/243 [==============================] - 0s 1ms/step - loss: 9.4203 - accuracy: 0.4897 - val_loss: 8.8612 - val_accuracy: 0.2857
Epoch 3/100
243/243 [==============================] - 0s 1ms/step - loss: 7.5139 - accuracy: 0.5597 - val_loss: 7.2584 - val_accuracy: 0.3214
Epoch 4/100
243/243 [==============================] - 0s 1ms/step - loss: 6.0437 - accuracy: 0.5761 - val_loss: 6.4001 - val_accuracy: 0.2500
Epoch 5/100
243/243 [==============================] - 0s 989us/step - loss: 5.1768 - accuracy: 0.5103 - val_loss: 5.0399 - val_accuracy: 0.1429
Epoch 6/100
243/243 [==============================] - 0s 957us/step - loss: 4.2625 - accuracy: 0.5638 - val_loss: 4.0407 - val_accuracy: 0.2500
Epoch 7/100
243/243 [==============================] - 0s 1ms/step - loss: 3.6266 - accurac

In [5]:
preds=model.predict_classes(X_test)
print(classification_report(y_test, preds, target_names=target_names_all))

              precision    recall  f1-score   support

  monoclinic       0.69      0.88      0.77        25
   triclinic       0.89      0.59      0.71        27
orthorhombic       0.89      1.00      0.94        16

    accuracy                           0.79        68
   macro avg       0.82      0.82      0.81        68
weighted avg       0.81      0.79      0.79        68



# 4. SVM

In [6]:
svm_model = SVC(kernel='linear')
svm_model.fit(train_data.drop(['Crystal System'], axis=1), train_data['Crystal System'])

# Make predictions on the test data
test_predictions = svm_model.predict(test_data.drop(['Crystal System'], axis=1))

print(classification_report(test_data['Crystal System'], test_predictions, target_names=target_names_all))

              precision    recall  f1-score   support

  monoclinic       0.42      0.72      0.53        25
   triclinic       0.68      0.63      0.65        27
orthorhombic       0.00      0.00      0.00        16

    accuracy                           0.51        68
   macro avg       0.37      0.45      0.39        68
weighted avg       0.42      0.51      0.45        68



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 5. LR

In [7]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
lr_model.fit(train_data.drop(['Crystal System'], axis=1), train_data['Crystal System'])


test_predictions = lr_model.predict(test_data.drop(['Crystal System'], axis=1))

print(classification_report(test_data['Crystal System'], test_predictions, target_names=target_names_all))

              precision    recall  f1-score   support

  monoclinic       0.42      0.60      0.49        25
   triclinic       0.64      0.59      0.62        27
orthorhombic       0.57      0.25      0.35        16

    accuracy                           0.51        68
   macro avg       0.54      0.48      0.49        68
weighted avg       0.54      0.51      0.51        68



/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# 6. naive_bayes

In [8]:
from sklearn.naive_bayes import BernoulliNB

bnb_model = BernoulliNB()
bnb_model.fit(train_data.drop(['Crystal System'], axis=1), train_data['Crystal System'])

test_predictions = bnb_model.predict(test_data.drop(['Crystal System'], axis=1))

print(classification_report(test_data['Crystal System'], test_predictions, target_names=target_names_all))

              precision    recall  f1-score   support

  monoclinic       0.37      0.60      0.45        25
   triclinic       0.52      0.52      0.52        27
orthorhombic       0.00      0.00      0.00        16

    accuracy                           0.43        68
   macro avg       0.29      0.37      0.32        68
weighted avg       0.34      0.43      0.37        68



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 7. Voting Model

In [9]:
from sklearn.ensemble import VotingClassifier


svm_model = SVC(kernel='linear')
lr_model = LogisticRegression()
bnb_model = BernoulliNB()

voting_model = VotingClassifier(estimators=[
    ('svm', svm_model),
    ('lr', lr_model),
    ('brb', bnb_model)
], voting='hard') 

voting_model.fit(train_data.drop(['Crystal System'], axis=1), train_data['Crystal System'])

test_predictions = voting_model.predict(test_data.drop(['Crystal System'], axis=1))

print(classification_report(test_data['Crystal System'], test_predictions, target_names=target_names_all))

              precision    recall  f1-score   support

  monoclinic       0.40      0.68      0.51        25
   triclinic       0.65      0.63      0.64        27
orthorhombic       0.00      0.00      0.00        16

    accuracy                           0.50        68
   macro avg       0.35      0.44      0.38        68
weighted avg       0.41      0.50      0.44        68



/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
